In [ ]:
%load_ext autoreload
%autoreload 2

In [62]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from eval_ds import *
from embed_evals import get_knn_score

In [5]:
df = pd.read_csv('../datasets/weevil_vespula/vw2d.relabelled.csv', index_col=0)
selected_features = ["bodyheadszratio", "bodyheadcolordiff"]
x, y = df[selected_features].values, df['pseudo_label'].values

In [6]:
from collections import OrderedDict
p2i_train = pickle.load(open('../datasets/weevil_vespula/p2i.pkl', 'rb'))
p2i_valid = pickle.load(open('../datasets/weevil_vespula/p2i_valid.pkl', 'rb'))
p2i_test = pickle.load(open('../datasets/weevil_vespula/p2i_test.pkl', 'rb'))
i2p_train = OrderedDict({v:int(k) for k, v in p2i_train.items()})
i2p_valid = OrderedDict({v:int(k) for k, v in p2i_valid.items()})
i2p_test = OrderedDict({v:int(k) for k, v in p2i_test.items()})
assert list(i2p_train.keys()) == list(range(len(i2p_train)))
assert list(i2p_valid.keys()) == list(range(len(i2p_valid)))
assert list(i2p_test.keys()) == list(range(len(i2p_test)))

In [24]:
x_train, y_train = x[list(i2p_train.values())], y[list(i2p_train.values())]
x_valid, y_valid = x[list(i2p_valid.values())], y[list(i2p_valid.values())]
x_test, y_test = x[list(i2p_test.values())], y[list(i2p_test.values())]
xdst = euc_dist(x_test, x_train)
xds = get_ds(xdst, y_test, y_train)

In [25]:
w = [1, 0]
h = trans(x, w)
h_train = trans(x_train, w)
h_valid = trans(x_valid, w)
h_test = trans(x_test, w)
hdst = euc_dist(h_test, h_train)
nino = get_ds(hdst, y_test, y_train)
choice = get_ds_choice(hdst, nino)
chosen = get_ds_chosen(choice, nino)
eval_ds_choice(choice, nino, y_test, y_train).mean()

0.7

In [23]:
emb, models, folds = {}, ['RESN'], ['train', 'valid', 'test']
for model in models:
    for fold in folds:
        if model not in emb:
            emb[model] = {}
        emb[model][fold] = pickle.load(open(f'../embeds/wv_2d/pretrained/{model}_{fold}_emb50.pkl', 'rb'))

In [52]:
r_test, r_train = emb['RESN']['test'], emb['RESN']['train']
rdst = euc_dist(r_test, r_train)
rds = get_ds(rdst, y_test, y_train)

In [59]:
[
    eval_ds(xdst, xds, y_test, y_train).mean(),
    eval_ds(rdst, rds, y_test, y_train).mean(),
    eval_ds(rdst, xds, y_test, y_train).mean(),
]


[1.0, 0.925, 0.9]

In [60]:
[
    get_knn_score(x_train, y_train, x_test, y_test),
    get_knn_score(r_train, y_train, r_test, y_test),
]


[1.0, 0.925]

In [ ]:
def get_ds_nifo(dist, y_test, y_train):
    mask_train = np.tile(y_train, (len(y_test), 1))
    apply_mask = lambda x, m: x + (-(m - 1) * x.max())
    ds = np.arange(len(y_test)).reshape(-1, 1)
    ni = np.arange(len(y_test)).reshape(-1, 1)
    fo = np.arange(len(y_test)).reshape(-1, 1)
    for label in np.unique(y_train):
        mask_in = label == mask_train
        mask_out = label == mask_train
        ni = np.argmin(apply_mask(dist, mask_in), 1)
        fo = np.argmax(apply_mask(dist, mask_out), 1)
        ni = np.hstack([ni, ni.reshape(-1, 1)])
        fo = np.hstack([fo, fo.reshape(-1, 1)])
    ds = ni
    return ds

In [63]:
mask_train = np.tile(y_train, (len(y_test), 1))